# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

import pandas as pd
import numpy as np
import re
import pickle

from scipy import stats
from scipy.stats import randint

from sqlalchemy import create_engine

from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk;nltk.download('popular');nltk.download('stopwords')

from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier

from xgboost import XGBClassifier

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


from sklearn.metrics import precision_recall_fscore_support

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/dirk/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/dirk/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/dirk/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/dirk/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/dirk/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/dirk/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!

In [4]:
# load data from database
engine = create_engine('sqlite:///../data/disaster.db')
df = pd.read_sql_table('Messages', engine)

X = df['message']
y = df.drop(columns=['message'])

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """_summary_

    Args:
        text (_type_): _description_

    Returns:
        _type_: _description_
    """
    # prep nltk transformation objects
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "URL")

    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    lemmed = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]


    return lemmed 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
                        ('vect', CountVectorizer(tokenizer = tokenize)),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultiOutputClassifier(DecisionTreeClassifier(random_state=42, max_depth=7, min_samples_split=5)))
                    ])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7feb39f47dc0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier(max_depth=7,
                                                                        min_samples_split=5,
                                                                        random_state=42)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_hat = pipeline.predict(X_test)

In [10]:
def report_f1_score_precision_recall(y_test, y_hat):
        """_summary_

        Args:
            y_test (_type_): _description_
            y_hat (_type_): _description_

        Returns:
            _type_: _description_
        """
        row_list = []
        for col_number, col_name in enumerate(y_test.columns):
                for label in np.unique(y_hat.ravel()):

                        # Get the precision, reacall, f_score and support for column number col_number
                        pre_rec_f1_supp = precision_recall_fscore_support(y_true = y_test.iloc[:,col_number], y_pred = y_hat[:,col_number])
                        # The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label a negative sample as positive.
                        # The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.
                        # The F-beta score can be interpreted as a weighted harmonic mean of the precision and recall, where an F-beta score reaches its best value at 1 and worst score at 0.
                        # The F-beta score weights recall more than precision by a factor of beta. beta == 1.0 means recall and precision are equally important.
                        # The support is the number of occurrences of each class in y_true.

                        # Collect the results. The dicts are collected in row_list which is converted to a Dataframe at the end.

                        result = { 'Label': col_name, 'Value': label, 'Precision': pre_rec_f1_supp[0][label], 'Recall': pre_rec_f1_supp[1][label],
                                'F-1 Score': pre_rec_f1_supp[2][label], 'Support': pre_rec_f1_supp[3][label] }
                        row_list.append(result)
        for a in ['micro', 'macro', 'weighted']:
                average = precision_recall_fscore_support(y_true = y_test, y_pred = y_hat, average=a)
                result = { 'Label': 'Average', 'Value': a, 'Precision': average[0], 'Recall': average[1],
                                'F-1 Score': average[2], 'Support': '-' }
                row_list.append(result)
        res_df=pd.DataFrame(row_list)
        res_df.set_index(['Label', 'Value'], inplace=True)
        return res_df

report_f1_score_precision_recall(y_test, y_hat)

Precision    Recall  F-1 Score Support
Category      Label                                           
related       0          0.479167  0.019008   0.036566    1210
              1          0.769961  0.993747   0.867657    3998
request       0          0.835441  0.979389   0.901706    4318
              1          0.390411  0.064045   0.110039     890
offer         0          0.995004  0.999228   0.997111    5182
...                           ...       ...        ...     ...
direct_report 0          0.812113  0.994314   0.894025    4221
              1          0.400000  0.016211   0.031159     987
Average       micro      0.720674  0.271667   0.394589       -
              macro      0.158851  0.045662   0.053415       -
              weighted   0.422777  0.271667   0.260445       -

[73 rows x 4 columns]

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
# define the grid of values to search
parameters = dict()
# clf__estimator__ prefix to address the model in the pipeline above

parameters['clf__estimator__max_depth'] = [5, 7, 9, 11]
parameters['clf__estimator__min_samples_split'] = [2, 5, 7, 9]


# define the evaluation procedure
cv = RepeatedKFold(n_splits=3, n_repeats=2, random_state=42)

# define the grid search procedure
grid_search = GridSearchCV(verbose=3, estimator=pipeline, param_grid=parameters, n_jobs=1, cv=cv, scoring='accuracy')

# execute the grid search
grid_result = grid_search.fit(X_train, y_train)

# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Fitting 6 folds for each of 12 candidates, totalling 72 fits
[CV 1/6] END clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=2;, score=0.186 total time=  11.1s
[CV 2/6] END clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=2;, score=0.192 total time=  11.6s
[CV 3/6] END clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=2;, score=0.176 total time=  11.2s
[CV 4/6] END clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=2;, score=0.175 total time=  11.0s
[CV 5/6] END clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=2;, score=0.186 total time=  11.3s
[CV 6/6] END clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=2;, score=0.197 total time=  11.3s
[CV 1/6] END clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=5;, score=0.186 total time=  11.2s
[CV 2/6] END clf__estimator__max_depth=2, clf__estimator__min_samples_leaf=5;, score=0.193 total time=  10.8s
[CV 3/6] END clf__estimator__max_depth=2, clf__estimator__m

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
y_hat2 = grid_result.best_estimator_.predict(X_test)

In [13]:
report_f1_score_precision_recall(y_test, y_hat2)

Precision    Recall  F-1 Score Support
Category      Label                                           
related       0          0.642857  0.022314   0.043131    1210
              1          0.771003  0.996248   0.869271    3998
request       0          0.829109  1.000000   0.906571    4318
              1          0.000000  0.000000   0.000000     890
offer         0          0.995008  1.000000   0.997498    5182
...                           ...       ...        ...     ...
direct_report 0          0.810759  0.999763   0.895396    4221
              1          0.666667  0.002026   0.004040     987
Average       micro      0.757543  0.259146   0.386184       -
              macro      0.115702  0.038907   0.042303       -
              weighted   0.360772  0.259146   0.239169       -

[73 rows x 4 columns]

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
# Use TfidfVectorizer instead of CountVectorizer
# Use XGB Classifier

xgb_pipeline = Pipeline([
                        ('vect', CountVectorizer(tokenizer = tokenize)),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultiOutputClassifier(XGBClassifier(random_state=42, eval_metric='logloss')))
                    ])


param_dist = {'clf__estimator__n_estimators': stats.randint(50, 300),
              'clf__estimator__max_depth': [3, 5, 7, 9],
             }

cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)

clf = RandomizedSearchCV(estimator=xgb_pipeline, 
                         param_distributions = param_dist,
                         cv = cv,  
                         error_score = 0, 
                         verbose = 3)


clf.fit(X_train, y_train)

xgb_y_hat = clf.predict(X_test)

report_f1_score_precision_recall(y_test, xgb_y_hat)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV 1/10] END clf__estimator__max_depth=7, clf__estimator__n_estimators=107;, score=0.141 total time= 1.7min
[CV 2/10] END clf__estimator__max_depth=7, clf__estimator__n_estimators=107;, score=0.154 total time= 1.7min
[CV 3/10] END clf__estimator__max_depth=7, clf__estimator__n_estimators=107;, score=0.140 total time= 1.6min
[CV 4/10] END clf__estimator__max_depth=7, clf__estimator__n_estimators=107;, score=0.145 total time= 1.7min
[CV 5/10] END clf__estimator__max_depth=7, clf__estimator__n_estimators=107;, score=0.142 total time= 1.7min
[CV 6/10] END clf__estimator__max_depth=7, clf__estimator__n_estimators=107;, score=0.141 total time= 1.8min
[CV 7/10] END clf__estimator__max_depth=7, clf__estimator__n_estimators=107;, score=0.136 total time= 1.8min
[CV 8/10] END clf__estimator__max_depth=7, clf__estimator__n_estimators=107;, score=0.149 total time= 1.8min
[CV 9/10] END clf__estimator__max_depth=7, clf__estimator__n_esti

KeyboardInterrupt: 

### 9. Export your model as a pickle file

In [ ]:

with open('disaster-response-model.pkl', 'wb') as file:
    pickle.dump(clf, file)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.